<span style="color: cyan;">

#### ```Portfolio Assignment 6.1: Feature Selection```

In this assignment, we will work with a dataset to build a linear regression model that predicts a target variable.

The dataset contains various features, and your goal is to select the most relevant ones for the model.

Let's start with importing what we need...  

In [ ]:
import pandas as pd 
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 

<span style="color: cyan;">

##### Let's load the publicly available diabetes dataset and print out a description of the dataset.<br>Your task is to build the best linear regression model you can, using this data to predict the 'target' field.

#### Diabetes dataset

Ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients, as well as the response of interest, a quantitative measure of disease progression one year after baseline.
Data Set Characteristics
Number of Instances: 442
Number of Attributes
First 10 columns are numeric predictive values
Target: Column 11 is a quantitative measure of disease progression one year after baseline
Attribute Information
 - age:     age in years <br>
  - sex:     sex <br>
  - bmi:     body mass index <br>
  - bp:      average blood pressure <br>
  - s1:      tc, total serum cholesterol <br>
  - s2:      ldl, low-density lipoproteins <br>
  - s3:      hdl, high-density lipoproteins <br>
  - s4:      tch, total cholesterol / HDL <br>
  - s5:      ltg, possibly log of serum triglycerides level <br>
  - s6:      glu, blood sugar level <br>
  - s7:      target, a quantitative measure of disease progression one year after baseline <br>
<hr>

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times n_samples (i.e. the sum of squares of each column totals 1).
<br>
For more information see:
Bradley Efron, Trevor Hastie, Iain Johnstone and Robert Tibshirani (2004) "Least Angle Regression," Annals of Statistics (with discussion), 407-499. (https://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)
<hr>
Source URL: https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html<br>
Data URL: https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt<br>
Note: The Data URL mentioned-above is obtained from the source URL. <br>The source URL provides detailed information about the dataset, variables and also reference links including the dataset link.

##### Read in data into a dataframe then print the dataframe head.

The first line uses pandas' `read_csv` function to load a dataset from a remote URL into a DataFrame named `df`. The `sep="\t"` argument specifies that the file is tab-delimited, meaning columns in the data are separated by tab characters rather than commas. This is important for correctly parsing the structure of the data.

The second line, `df.head()`, displays the first five rows of the DataFrame. This is a common practice to quickly inspect the data after loading, allowing you to verify that the import worked correctly and to get an initial sense of the dataset's contents and column names. This step is especially useful when working with new or unfamiliar datasets.

In [2]:
df = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep="\t")
df.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


<span style="color: cyan;">

#### Basic field information

The line `df.info()` calls a pandas DataFrame method that prints a concise summary of the DataFrame's structure. This summary includes the number of rows and columns, the column names, the data types of each column, and the number of non-null (non-missing) values in each column. Using `df.info()` is a quick way to assess the overall shape and health of your dataset. It helps you identify which columns contain missing data, what types of data are present (such as integers, floats, or objects), and whether any columns might need cleaning or conversion before analysis. This step is especially useful early in a data analysis workflow to guide further preprocessing and feature engineering.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AGE     442 non-null    int64  
 1   SEX     442 non-null    int64  
 2   BMI     442 non-null    float64
 3   BP      442 non-null    float64
 4   S1      442 non-null    int64  
 5   S2      442 non-null    float64
 6   S3      442 non-null    float64
 7   S4      442 non-null    float64
 8   S5      442 non-null    float64
 9   S6      442 non-null    int64  
 10  Y       442 non-null    int64  
dtypes: float64(6), int64(5)
memory usage: 38.1 KB


<span style="color: cyan;">

#### Convert sex to a categorical variable

The first line converts the `SEX` column in the DataFrame `df` to a categorical data type using pandas' `astype('category')` method. This is useful for columns that represent discrete groups or labels, such as gender, because it allows for more efficient storage and enables category-specific operations.

The second line retrieves the unique values present in the `SEX` column with `df.SEX.unique()`, and then sorts these values using `sort_values()`. This provides a sorted list of all distinct categories in the `SEX` column, which is helpful for quickly understanding the possible values and ensuring data consistency before further analysis or modeling.

In [4]:
df['SEX'] = df['SEX'].astype('category')
df.SEX.unique().sort_values()

[1, 2]
Categories (2, int64): [1, 2]

<span style="color: cyan;">

#### Basic field information

The line `df.info()` displays a concise summary of the DataFrame `df` in pandas. This summary includes the number of rows and columns, the names and data types of each column, and the count of non-null (non-missing) values per column. 

Using `df.info()` is a quick way to check the overall structure and health of your dataset. It helps you identify missing data, understand which columns are categorical or numerical, and spot any potential issues before further analysis or modeling. This step is especially useful early in a data analysis workflow to guide data cleaning and preprocessing decisions.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   AGE     442 non-null    int64   
 1   SEX     442 non-null    category
 2   BMI     442 non-null    float64 
 3   BP      442 non-null    float64 
 4   S1      442 non-null    int64   
 5   S2      442 non-null    float64 
 6   S3      442 non-null    float64 
 7   S4      442 non-null    float64 
 8   S5      442 non-null    float64 
 9   S6      442 non-null    int64   
 10  Y       442 non-null    int64   
dtypes: category(1), float64(6), int64(4)
memory usage: 35.2 KB


<span style="color: cyan;">

#### Next, examine the dataframe

The first line uses pandas' `describe` method with the argument `include='all'` to generate a summary of the DataFrame `df`. This summary includes descriptive statistics for all columns, regardless of their data type (numeric, categorical, or object). For numeric columns, it provides metrics like count, mean, standard deviation, minimum, and maximum values. For categorical columns, it shows the number of unique values, the most frequent value, and its frequency.

The second line, `display(dfDescription)`, outputs this summary in a readable format, especially useful in Jupyter notebooks. This step helps you quickly understand the distribution, central tendency, and variability of your data, as well as spot potential issues such as missing values or unexpected categories. It's a valuable tool for initial data exploration and quality assessment.

In [6]:
dfDescription = df.describe(include='all')
display(dfDescription)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
count,442.000000,442.0,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
unique,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,48.518100,NaN,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,NaN,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,NaN,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,NaN,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,NaN,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,NaN,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000


<span style="color: cyan;">

#### Split dataframe into train and test subsets

This line uses the `train_test_split` function from scikit-learn to divide the DataFrame `df` into two subsets: `df_train` and `df_test`. The `test_size=0.3` argument specifies that 30% of the data should be allocated to the test set, while the remaining 70% goes to the training set. The `random_state=42` parameter sets a seed for the random number generator, ensuring that the split is reproducible and yields the same result each time the code is run. Why 42? It's a common convention in data science to use 42 as a random seed, often referenced in popular culture (e.g., "The Hitchhiker's Guide to the Galaxy") as the "Answer to the Ultimate Question of Life, the Universe, and Everything."

Splitting the data in this way is a standard practice in machine learning workflows. The training set is used to fit the model, while the test set is reserved for evaluating the model's performance on unseen data. This helps prevent overfitting and provides a more realistic estimate of how the model will perform in real-world scenarios.

In [7]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

<span style="color: cyan;">

#### Fit Multilinear OLS regression model using training dataset and save the result in 'est_train' variable. 

This code fits a multiple linear regression model using the training subset of your data. First, `X = df_train.drop(columns=['Y'])` creates a new DataFrame `X` containing all columns except `'Y'`, which is the target variable you want to predict. The line `y = df_train['Y']` extracts the target variable into a separate Series `y`.

Next, `X = sm.add_constant(X)` adds a constant (intercept) term to the predictors, which is necessary for most regression models to estimate the baseline value when all features are zero. The line `est_train = sm.OLS(y, X).fit()` fits an Ordinary Least Squares (OLS) regression model using the statsmodels (sm) library, with `y` as the dependent variable and `X` as the independent variables.

Finally, `display(est_train.summary())` shows a detailed summary of the fitted model, including coefficients, statistical significance, goodness-of-fit metrics, and diagnostic information. This summary helps you interpret the model's results and assess which features are most influential in predicting the target variable.

In [8]:
X         = df_train.drop(columns=['Y'])
y         = df_train['Y']
X         = sm.add_constant(X)
est_train = sm.OLS(y, X).fit()
display(est_train.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.508
Method:                 Least Squares   F-statistic:                     32.86
Date:                Wed, 23 Jul 2025   Prob (F-statistic):           1.37e-42
Time:                        12:01:49   Log-Likelihood:                -1671.5
No. Observations:                 309   AIC:                             3365.
Df Residuals:                     298   BIC:                             3406.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -316.2886     79.210     -3.993      0.000    -472.171    -160.406
AGE            0.1063      0.270      0.394      0.694      -0.425       0.637
SEX          -24.9463      6.987     -3.570      0.000     -38.697     -11.196
BMI            5.8881      0.877      6.717      0.000       4.163       7.613
BP             1.3372      0.268      4.998      0.000       0.811       1.864
S1            -1.2411      0.659     -1.882      0.061      -2.539       0.057
S2             0.7935      0.604      1.314      0.190      -0.395       1.982
S3             0.4460      0.904      0.493      0.622      -1.333       2.226
S4            10.6288      7.255      1.465      0.144      -3.649      24.906
S5            60.0963     18.952      3.171      0.002      22.800      97.393
S6             0.1714      0.330      0.519      0.604      -0.479       0.822
==============================================================================
Omnibus:                        1.511   Durbin-Watson:                   1.764
Prob(Omnibus):                  0.470   Jarque-Bera (JB):                1.417
Skew:                           0.056   Prob(JB):                        0.492
Kurtosis:                       2.688   Cond. No.                     7.03e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.03e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<span style="color: cyan;">

#### Extract non significant coef (p< .05: SEX + BMI + S3 + S5), rerun model.

The first line creates a new DataFrame `X` by dropping the columns `'SEX'`, `'BMI'`, `'S3'`, and `'S5'` from the original DataFrame `df`. This is typically done to exclude these features from further analysis or modeling, possibly because they were found to be less significant in previous studies.

The second line splits the DataFrame `df` into training and test subsets using scikit-learn's `train_test_split` function. Here, 30% of the data is allocated to the test set and 70% to the training set, with `random_state=42` ensuring reproducibility.

The third line fits a multiple linear regression model using the statsmodels formula API. The formula `"Y ~ SEX + BMI + S3 + S5"` specifies that the target variable `Y` should be predicted using the features `'SEX'`, `'BMI'`, `'S3'`, and `'S5'` from the training data. The `.fit()` method trains the model.

Finally, `display(est_train.summary())` outputs a detailed summary of the fitted regression model, including coefficients, statistical significance, and model diagnostics. This helps you evaluate the impact of the selected features on the target variable and assess the model's overall performance.

In [9]:
X                 = df.drop(columns=['SEX', 'BMI', 'S3', 'S5'])
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)
est_train         = ols(formula="Y ~  SEX + BMI  + S3 + S5 ", data=df_train).fit()
display(est_train.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     67.34
Date:                Wed, 23 Jul 2025   Prob (F-statistic):           9.47e-41
Time:                        12:01:49   Log-Likelihood:                -1688.3
No. Observations:                 309   AIC:                             3387.
Df Residuals:                     304   BIC:                             3405.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -176.6489     42.698     -4.137      0.000    -260.669     -92.629
SEX[T.2]     -17.1853      7.019     -2.448      0.015     -30.998      -3.372
BMI            7.3777      0.852      8.659      0.000       5.701       9.054
S3            -1.0659      0.302     -3.526      0.000      -1.661      -0.471
S5            41.8242      7.239      5.778      0.000      27.580      56.069
==============================================================================
Omnibus:                        2.800   Durbin-Watson:                   1.837
Prob(Omnibus):                  0.247   Jarque-Bera (JB):                2.385
Skew:                           0.109   Prob(JB):                        0.303
Kurtosis:                       2.629   Cond. No.                         756.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<span style="color: cyan;">

#### How well does it do on the test data? Lets use the model we trained on the training data to make predictions on the test data and then measure the R^2

The first line uses the trained regression model (`est_train`) to make predictions on the test dataset (`df_test`). The `predict` method generates predicted values for the target variable based on the features in the test set.

The second line calculates the R-squared (coefficient of determination) score using scikit-learn's `r2_score` function. This metric compares the actual target values (`df_test['Y']`) to the predicted values (`test_pred`) and quantifies how well the model explains the variance in the test data. An R-squared value closer to 1 indicates better predictive performance.

Finally, `display('OOS R-squared: ' + str(r2))` outputs the out-of-sample R-squared value, providing a clear summary of the model's performance on unseen data. This step is essential for evaluating how well the model generalizes beyond the training set.

In [10]:
test_pred = est_train.predict(df_test)
r2        = r2_score(df_test['Y'], test_pred)
display('OOS R-squared: ' + str(r2))

'OOS R-squared: 0.4851185328484513'

<span style="color: cyan;">

#### Testing the model with new unseen data


This code demonstrates how to use a trained regression model to make predictions on new, unseen data. First, a new DataFrame `new_data` is created with specific values for the features `'SEX'`, `'BMI'`, `'S3'`, and `'S5'`. These values represent a hypothetical patient or observation for which you want to predict the target variable.

Next, `sm.add_constant(new_data)` adds an intercept term to the new data, ensuring it matches the format expected by the regression model. The line `prediction = est_train.predict(new_data)` uses the trained model (`est_train`) to generate a prediction for the target variable based on the provided feature values.

Finally, `display('Prediction for new data: ' + str(prediction.iloc[0]))` outputs the predicted value, allowing you to see the model's estimate for this new observation. This process is useful for applying your model to real-world scenarios or testing its behavior with different input values.

In [11]:
new_data = pd.DataFrame({
    'SEX': [1],   
    'BMI': [29.0],
    'S3': [0.5],  
    'S5': [0.3]  
})
new_data   = sm.add_constant(new_data)
prediction = est_train.predict(new_data)
# Display the prediction for the new data, which is the first new data point, and it is very low.
display('Prediction for new data: ' + str(prediction.iloc[0]))
new_data1 = pd.DataFrame({
    'SEX': [2],   
    'BMI': [36.0],
    'S3': [0.5], 
    'S5': [0.3]
})
new_data1   = sm.add_constant(new_data1)
prediction1 = est_train.predict(new_data1)
# Display the prediction for the new data, which is the second new data point, and it is very high.
display('Prediction for new data: ' + str(prediction1.iloc[0]))

'Prediction for new data: 49.31753104199821'

'Prediction for new data: 83.77587802403008'